## Model Saving/Loading Example

This set of examples shows how to load and import template models in order to make setting up and reproducing circuit fits easier.

In [ ]:
# Load libraries

import impedance.models.circuits as circuits
import impedance.preprocessing as preprocessing
import matplotlib.pyplot as plt
import numpy as np
from impedance.visualization import plot_nyquist

In [ ]:
# Load data from the example EIS result
frequencies, Z = preprocessing.readCSV("../../../data/exampleData.csv")

# keep only the impedance data in the first quandrant
frequencies, Z = preprocessing.ignoreBelowX(frequencies, Z)

### Example 1. Importing and Exporting Models

Call the `circuit.save()` function to export the model to a human readable JSON file. The following code generates a test circuit and export it as a template. Here we are using an unfitted model as a template.


In [ ]:
test_circuit = circuits.CustomCircuit(
    initial_guess=[0.01, 0.005, 0.1, 0.005, 0.1, 0.001, 200],
    circuit="R0-p(R1,C1)-p(R2,C2)-Wo1",
)

print(test_circuit)

test_circuit.save("template_model.json")

Call the `model_io.model_import` function to import the model back as a template.

In [ ]:
loaded_template = circuits.CustomCircuit()
loaded_template.load("template_model.json")

print("Loaded Template")
print(loaded_template)

### Example 2. Using imported template model to fit data

After the model has been imported as a template, it can be used as a starting point to fit data. This saves on needing to configure the initial parameters each time a fit is performed and to persist starting conditions across several fitting sessions.

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
f_pred = np.logspace(5, -2)
loaded_template.fit(frequencies, Z)

imported_circuit_init = loaded_template.predict(f_pred, use_initial=True)
imported_circuit_fit = loaded_template.predict(f_pred)

plot_nyquist(ax, Z)
plot_nyquist(ax, imported_circuit_init)
plot_nyquist(ax, imported_circuit_fit)

ax.legend(["Data", "Loaded Template Initial", "Loaded Template Fit"])

plt.show()
print(loaded_template)

### Example 3. Using fitted data as a starting point for new fits

Consider the case where a successful fit has been performed and a new set of EIS data is obtained which is similar to the first spectrum. It is useful to use the successfully fitted parameters as a starting point for subsequent fits.

In [ ]:
# Export the fitted model as a template

loaded_template.save("fitted_template.json")

Using the exported model's fitted parameters, generate a new circuit using the fitted parameters as initial guesses by suppling the `fitted_as_initial` parameter as `True`.

In [ ]:
fitted_template = circuits.CustomCircuit()
fitted_template.load("fitted_template.json", fitted_as_initial=True)
print(fitted_template)

Z2 is a similar impedance spectra that we can fit using the previous fitted parameters as starting points. It has been shifted by 5 mOhm in the real axis and the data has been scaled by 1.5x.

In [ ]:
Z2 = (0.005 + Z.real) * 1.5 + 1.5j * Z.imag

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
f_pred = np.logspace(5, -2)
fitted_template.fit(frequencies, Z2)

imported_circuit_init = fitted_template.predict(f_pred, use_initial=True)
imported_circuit_fit = fitted_template.predict(f_pred)

plot_nyquist(ax, Z)
plot_nyquist(ax, Z2)
plot_nyquist(ax, imported_circuit_init)
plot_nyquist(ax, imported_circuit_fit)

ax.legend(["Original Data", "Shifted Data", "Template Initial", "Loaded Template Fit"])

plt.show()

print(fitted_template)